## 1 - Importações

In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
df = pd.read_csv(
    'PETR4_textuais_numericos.csv',
    encoding='utf-8-sig', 
    index_col='date'
)

In [3]:
df.shape

(3622, 13)

In [4]:
df.columns

Index(['close', 'compound_gn', 'negative_gn', 'neutral_gn', 'positive_gn',
       'compound_tw', 'negative_tw', 'neutral_tw', 'positive_tw', 'open',
       'high', 'low', 'volume'],
      dtype='object')

## 2 - Funções auxiliares

### 2.1 Calcular os Indicadores

In [5]:
import pandas as pd

def calcular_mms(df, periodo):
    """
    Calcula a Média Móvel Simples (MMS).
    
    Args:
        precos (list ou pd.Series): Série de preços.
        periodo (int): Período da média móvel.
    
    Returns:
        pd.Series: Média Móvel Simples.
    """
    df[f'mms_{periodo}'] = df['close'].rolling(window=periodo).mean()
    
    return df

def calcular_mme(df, periodo):
    """
    Calcula a Média Móvel Exponencial (MME).
    
    Args:
        precos (list ou pd.Series): Série de preços.
        periodo (int): Período da média móvel.
    
    Returns:
        pd.Series: Média Móvel Exponencial.
    """
    df[f'mme_{periodo}'] = df['close'].ewm(span=periodo, adjust=False).mean()
    
    return df

def calcular_ifr(df, periodo=14):
    """
    Calcula o Índice de Força Relativa (IFR ou RSI - Relative Strength Index).
    
    Args:
        precos (list ou pd.Series): Série de preços.
        periodo (int): Período do IFR.
    
    Returns:
        pd.Series: Valores do IFR.
    """
    precos = df['close']
    variacao = precos.diff(1)
    
    ganho = variacao.where(variacao > 0, 0)
    perda = -variacao.where(variacao < 0, 0)
    
    media_ganho = ganho.rolling(window=periodo).mean()
    media_perda = perda.rolling(window=periodo).mean()
    
    rs = media_ganho / media_perda
    ifr = 100 - (100 / (1 + rs))
    df[f'ifr_{periodo}'] = ifr
    
    return df

### 2.2 Carregar Dados

In [6]:
# Exemplo de função para obter os dados das ações
def get_stock_data(stock):
    """
    Função para carregar os dados uma ação.

    Parâmetros:
        stock (str): Nome da ação.

    Retorna:
        tuple: DataFrame com os dados da ação e o nome da coluna de destino (target).
    """
    # Carregando
    df = pd.read_csv(
    f'{stock}_textuais_numericos.csv',
    encoding='utf-8-sig', 
    index_col='date'
    )

    # Calcular os indicadores técnicos
    df = calcular_mms(df, 30)
    df = calcular_mms(df, 5)
    df = calcular_mme(df, 5)
    df = calcular_mme(df, 30)
    df = calcular_ifr(df)

    # Limpar os dados
    df.dropna(inplace=True)

    return df

In [7]:
def load_stocks_data(stocks):
    """
    Carrega os dados das empresas usando a função get_stock_data e armazena-os em um dicionário.

    Parâmetros:
        stocks (list): Lista de códigos das ações.

    Retorna:
        dict: Dicionário com os códigos das ações como chave e os dados como valor.
    """
    stock_data_dict = {}
    
    for stock in stocks:
        try:
            print(f"Carregando dados para: {stock}")
            stock_data = get_stock_data(stock)  # Obtém os dados da função
            stock_data_dict[stock] = stock_data
        except Exception as e:
            print(f"Erro ao carregar dados para {stock}: {e}")
    
    return stock_data_dict

### 2.3 Combinações dos Dados

In [8]:
def make_combination_data(df, num_combination):
    """
    Combina dados com base em um identificador e retorna o DataFrame combinado e uma descrição.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados originais.
        num_combination (int): Número identificador da combinação.

    Retorna:
        tuple: DataFrame combinado e descrição da combinação.
    """
    # Definindo as combinações
    # Stock Data
    stock_data = ['open','close','high','low','volume']
    # Google News
    google_news = ['compound_gn', 'negative_gn', 'neutral_gn', 'positive_gn']
    # Twitter
    twitter = ['compound_tw', 'negative_tw', 'neutral_tw', 'positive_tw']
    # MMS
    mms = ['mms_30', 'mms_5']
    # MME
    mme = ['mme_5', 'mme_30']
    # IFR de 14 periodos
    ifr = ['ifr_14']
    # Valor de fechamento
    close = ['close']
    
    # Mapear combinações para descrição e colunas
    combinations = {
        1: ("Stock Data", stock_data),
        2: ("Stock Data + Google News", stock_data + google_news),
        3: ("Stock Data + Twitter", stock_data + twitter),
        4: ("Stock Data + IT (IFR + MMS + MME)", stock_data + ifr + mms + mme),
        5: ("Google News + Twitter + IFR + MMS", close + google_news + twitter + ifr + mms),
        6: ("Google News + Twitter + IFR + MME", close + google_news + twitter + ifr + mme),
        7: ("Google News + Twitter + IFR + MME + MMS", close + google_news + twitter + ifr + mme + mms),
    }

    # Verifica se a combinação existe
    if num_combination not in combinations:
        raise ValueError(f"Combinação {num_combination} não é válida. Escolha entre 1 e {len(combinations)}.")

    # Recupera descrição e colunas da combinação
    description_combination, selected_columns = combinations[num_combination]

    # Retorna o DataFrame filtrado e a descrição
    return df[selected_columns], description_combination

### 2.4 Definição do Alvo - Regressão

In [9]:
def define_target(df, target_column='close'):
    """
    Define a coluna de alvo para um modelo de regressão com base no preço de fechamento do próximo dia.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna que será usada como base para o cálculo do alvo. Default é 'close'.
    
    Retorna:
        pd.DataFrame: DataFrame atualizado com a nova coluna de alvo ('close_next_day') e sem a última linha.
    """
    # Trabalhar com uma cópia explícita do DataFrame para evitar avisos
    df = df.copy()
    
    # Criar a coluna de preço de fechamento do próximo dia
    df['close_next_day'] = df[target_column].shift(-1)
    
    # Remover a última linha, pois não há valor para a previsão do próximo dia
    df = df[:-1]
 
    return df, 'close_next_day'

### 2.5 Treinamento dos modelos

In [10]:
def train_lstm_model(df, target_column, name_model):
    """
    Treina um modelo LSTM em dados de séries temporais e retorna as métricas de desempenho.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna de destino (target).

    Retorna:
        dict: Dicionário com métricas de desempenho (Loss, MSE, RMSE, MAE, MAPE, Acurácia).
    """
    # Separando as variáveis independentes (X) e dependente (y)
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    # Dividindo os dados em 80% para treino e 20% para teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Escalonando os dados para que os valores estejam entre 0 e 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Reshape dos dados para LSTM (adicionar dimensão para timesteps)
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    # Mapeamento de nomes de modelos para funções
    model_mapping = {
    'model_1': model_1,
    'model_2': model_2,
    'model_3': model_3,
    'model_linear_simple': model_linear_simple,
    }

    # Chamando o modelo
    if name_model == 'model_3':
        model = model_mapping[name_model]((X.shape[1],), learning_rate=0.001)
    else:
        model = model_mapping[name_model]((X_train_scaled.shape[1], X_train_scaled.shape[2]), learning_rate=0.001)
    
    # Configurando EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Treinando o modelo
    history = model.fit(
        X_train_scaled, y_train,
        #X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test_scaled, y_test),
        callbacks=[early_stopping],
        verbose=1
    )

    # Prevendo no conjunto de teste
    y_pred = model.predict(X_test_scaled)

    # Calculando métricas adicionais
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Coeficiente de determinação (R²)
    ss_total = np.sum((y_test - np.mean(y_test)) ** 2)  # Soma dos quadrados totais
    ss_residual = np.sum((y_test - y_pred.flatten()) ** 2)  # Soma dos quadrados residuais
    r2 = 1 - (ss_residual / ss_total)

    # Acurácia considerando 2% de margem de erro
    margin = 0.02
    accuracy = accuracy_score(
        (np.abs(y_test - y_pred.flatten()) <= margin * y_test).astype(int),
        np.ones_like(y_test)
    )

    # Montando o dicionário de métricas
    metrics_dict = {
        "Loss": history.history['loss'][-1],
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "R²": r2,  # Adicionando o R²
        "Acurácia": accuracy
    }

    return metrics_dict

In [11]:
def train_model_linear(df, target_column):
    """
    Treina um modelo de Regressão Linear Simples em dados de séries temporais e retorna as métricas de desempenho.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna de destino (target).

    Retorna:
        dict: Dicionário com métricas de desempenho (Loss, MSE, RMSE, MAE, MAPE, Acurácia).
    """
    # Separando as variáveis independentes (X) e dependente (y)
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    # Dividindo os dados em 80% para treino e 20% para teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Escalonando os dados para que os valores estejam entre 0 e 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Chamando a função para criar o modelo de Regressão Linear Simples
    model = model_linear_simple((X_train_scaled.shape[1],))

    # Treinando o modelo
    history = model.fit(
        X_train_scaled, y_train,
        epochs=50,
        batch_size=32,
        verbose=1,
        validation_data=(X_test_scaled, y_test)
    )

    # Prevendo no conjunto de teste
    y_pred = model.predict(X_test_scaled)

    # Calculando métricas adicionais
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Coeficiente de determinação (R²)
    ss_total = np.sum((y_test - np.mean(y_test)) ** 2)  # Soma dos quadrados totais
    ss_residual = np.sum((y_test - y_pred.flatten()) ** 2)  # Soma dos quadrados residuais
    r2 = 1 - (ss_residual / ss_total)

    # Acurácia considerando 2% de margem de erro
    margin = 0.02
    accuracy = accuracy_score(
        (np.abs(y_test - y_pred.flatten()) <= margin * y_test).astype(int),
        np.ones_like(y_test)
    )

    # Montando o dicionário de métricas
    metrics_dict = {
        "Loss": history.history['loss'][-1],
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "R²": r2,  # Adicionando o R²
        "Acurácia": accuracy
    }

    return metrics_dict

### 2.6 Treinamento de Todos Modelos

In [12]:
def train_model_all(df, target_column, name_model):
    """
    Treina um modelo LSTM em dados de séries temporais e retorna as métricas de desempenho.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna de destino (target).

    Retorna:
        dict: Dicionário com métricas de desempenho (Loss, MSE, RMSE, MAE, MAPE, Acurácia).
    """
    # Separando as variáveis independentes (X) e dependente (y)
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    # Dividindo os dados em 80% para treino e 20% para teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Escalonando os dados para que os valores estejam entre 0 e 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Reshape dos dados para LSTM (adicionar dimensão para timesteps)
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    # Mapeamento de nomes de modelos para funções
    model_mapping = {
    'model_1': model_1,
    'model_2': model_2,
    'model_3': model_3,
    'model_linear_simple': model_linear_simple,
    }

    # Chamando o modelo
    if name_model == 'model_3' or name_model == 'model_linear_simple':
        model = model_mapping[name_model]((X_train.shape[1],))
    
        # Treinando o modelo
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            verbose=1,
            validation_data=(X_test, y_test)
        )
        # Prevendo no conjunto de teste
        y_pred = model.predict(X_test)
    else:
        model = model_mapping[name_model]((X_train_scaled.shape[1], X_train_scaled.shape[2]), learning_rate=0.001)
            # Configurando EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
        # Treinando o modelo
        history = model.fit(
            X_train_scaled, y_train,
            #X_train, y_train,
            epochs=50,
            batch_size=32,
            validation_data=(X_test_scaled, y_test),
            callbacks=[early_stopping],
            verbose=1
        )
        # Prevendo no conjunto de teste
        y_pred = model.predict(X_test_scaled)
    
    # Calculando métricas adicionais
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Coeficiente de determinação (R²)
    ss_total = np.sum((y_test - np.mean(y_test)) ** 2)  # Soma dos quadrados totais
    ss_residual = np.sum((y_test - y_pred.flatten()) ** 2)  # Soma dos quadrados residuais
    r2 = 1 - (ss_residual / ss_total)

    # Acurácia considerando 2% de margem de erro
    margin = 0.02
    accuracy = accuracy_score(
        (np.abs(y_test - y_pred.flatten()) <= margin * y_test).astype(int),
        np.ones_like(y_test)
    )

    # Montando o dicionário de métricas
    metrics_dict = {
        "Loss": history.history['loss'][-1],
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "R²": r2,  # Adicionando o R²
        "Acurácia": accuracy
    }

    return metrics_dict

## 3 - Definição dos Modelos

### 3.1 Modelo 1

In [14]:
# MODEL 1

def model_1(input_shape, learning_rate=0.001):
    """
    Cria e compila um modelo LSTM.

    Parâmetros:
        input_shape (tuple): A forma da entrada (timesteps, features).
        learning_rate (float): Taxa de aprendizado para o otimizador Adam.

    Retorna:
        keras.Model: Modelo LSTM compilado.
    """
    # Criando o modelo com o uso explícito de Input
    model = Sequential([
        Input(shape=input_shape),
        LSTM(16, return_sequences=False),
        Dense(1)
    ])

    # Compilando o modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mse', 'mae', 'mape']
    )

    return model

### 3.2 Modelo 2

In [15]:
# MODEL 2

def model_2(input_shape, learning_rate=0.001, dropout_rate=0.03):
    """
    Cria e compila um modelo LSTM complexo com várias camadas e dropout.

    Parâmetros:
        input_shape (tuple): A forma da entrada (timesteps, features).
        learning_rate (float): Taxa de aprendizado para o otimizador Adam.
        dropout_rate (float): Taxa de dropout aplicada nas últimas camadas.

    Retorna:
        keras.Model: Modelo LSTM compilado.
    """
    # Criando o modelo com o uso explícito de Input
    model = Sequential([
        Input(shape=input_shape),
        LSTM(200, activation='tanh', recurrent_activation='sigmoid', return_sequences=True),
        LSTM(300, activation='tanh', recurrent_activation='sigmoid', return_sequences=True),
        LSTM(400, activation='tanh', recurrent_activation='sigmoid', return_sequences=True),
        Dropout(dropout_rate),  # Dropout na penúltima camada LSTM
        LSTM(400, activation='tanh', recurrent_activation='sigmoid', return_sequences=False),
        Dropout(dropout_rate),  # Dropout na última camada LSTM
        Dense(1)  # Camada de saída
    ])

    # Compilando o modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mse', 'mae', 'mape']
    )

    return model

### 3.3 Modelo 3

In [16]:
# Modelo 3
def model_3(input_shape):
    """
    Cria e compila um modelo denso com 3 camadas.

    Parâmetros:
        input_shape (tuple): A forma da entrada (features,).
        learning_rate (float): Taxa de aprendizado para o otimizador Adam.

    Retorna:
        keras.Model: Modelo denso compilado.
    """
    learning_rate=0.001
    
    # Criando o modelo
    model = Sequential([
        Input(shape=input_shape),
        Dense(32, activation='relu'),  # Primeira camada Dense
        Dense(8, activation='relu'),                            # Segunda camada Dense
        Dense(1)                                                # Camada de saída
    ])

    # Compilando o modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mse', 'mae', 'mape']
    )

    return model

### 3.4 Modelo 4 - model_linear_simple

In [17]:
# MODEL: Regressão Linear Simples

def model_linear_simple(input_shape):
    """
    Cria um modelo de Regressão Linear Simples.

    Parâmetros:
        input_shape (tuple): A forma da entrada (features,).

    Retorna:
        keras.Model: Modelo de regressão linear simples compilado.
    """
    from keras.models import Sequential
    from keras.layers import Dense

    # Criando o modelo
    model = Sequential([
        Input(shape=input_shape),
        Dense(1, activation='linear')  # Saída linear para regressão
    ])

    # Compilando o modelo
    model.compile(
        optimizer='adam',  # Otimizador Adam padrão
        loss='mean_squared_error',  # Perda de erro médio quadrático
        metrics=['mse', 'mae', 'mape']  # Métricas úteis para regressão
    )

    return model

## 4 - Treino dos modelos

### 4.1 Função Principal - process_stocks_and_save_metrics

In [19]:
def process_stocks_and_save_metrics(stock_list, num_combination, name_model):
    """
    Processa uma lista de ações, treina o modelo LSTM para cada uma e salva as métricas em um DataFrame.

    Parâmetros:
        stock_list (list): Lista de ações (str).
        get_stock_data_func (function): Função para obter os dados de cada ação.
            Deve retornar um DataFrame com os dados da ação e o nome da coluna de destino (target).

    Retorna:
        pd.DataFrame: DataFrame contendo o nome da ação, descrição dos dados, nome do modelo e as métricas.
    """
    results = []

    for stock in stock_list:
        print(f"{name_model}: Processando ação: {stock}, combination: {num_combination}...")

        # Obtém os dados da ação
        df = all_stock_data[stock]
        # Realizar as combination
        df, description_combination = make_combination_data(df, num_combination)
        # Definir o alvo
        df, target_column = define_target(df)

        # Treina o modelo e coleta as métricas
        metrics = train_lstm_model(df, target_column, name_model)
        #metrics = train_model_all(df, target_column, name_model)
        #metrics = train_model_windows(df, target_column, name_model)
        #metrics = train_model_linear(df, target_column)
        #metrics = train_model_sliding_window(df, target_column, name_model) 

        # Salva os resultados no formato desejado
        results.append({
            "Stock": stock,
            "Description": description_combination,
            "Model": name_model,
            **metrics
        })

    # Retorna os resultados em um DataFrame
    return pd.DataFrame(results)

    # Normalização
    # Normalizar os colunas
    #columns = ['close','open', 'high', 'low', 'mms_5', 'mms_30', 'mme_5','mme_30']
    
    # Normalizar
    #df = normalize_columns_2(df, columns, 30)
    
    return df, 'close_next_day', description_combination, 

### 4.2 Rodando em todas as Empresas

In [20]:
# Processando as ações com barra de progresso
'''
Opções de modelos
1: model_1
2: model_2
3: model_3
4: model_linear_simple

'''

# Lista de ações
stock_list = ['PETR4', 'VALE3', 'BBDC4','ITUB4']
#stock_list = ['PETR4']

# Carregar os dados
all_stock_data = load_stocks_data(stock_list)

name_model = 'model_1'
results_list = []

# Usando tqdm para exibir progresso
for i in tqdm(range(7), desc="Processando combinações"):
    results_list.append(process_stocks_and_save_metrics(stock_list, i + 1, name_model))
    #results_list.append(process_stocks_and_save_metrics(stocks, get_stock_data, i + 1, name_model))

# Combina os resultados em um único DataFrame
results_df = pd.concat(results_list, ignore_index=True)

# Exibir o resultado
results_df
# Salvar em CSV
results_df.to_csv(f'Estudos_IEEE_{name_model}.csv', sep=';', encoding='utf-8-sig')

Carregando dados para: PETR4
Carregando dados para: VALE3
Carregando dados para: BBDC4
Carregando dados para: ITUB4


Processando combinações:   0%|                                                                   | 0/7 [00:00<?, ?it/s]

model_1: Processando ação: PETR4, combination: 1...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1411 - mae: 0.3238 - mape: 18916.8555 - mse: 0.1411 - val_loss: 0.0313 - val_mae: 0.1643 - val_mape: 33.6240 - val_mse: 0.0313
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0128 - mae: 0.0877 - mape: 10751.6631 - mse: 0.0128 - val_loss: 0.0023 - val_mae: 0.0405 - val_mape: 9.2227 - val_mse: 0.0023
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 - mae: 0.0564 - mape: 1114.5963 - mse: 0.0049 - val_loss: 0.0015 - val_mae: 0.0322 - val_mape: 7.1523 - val_mse: 0.0015
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0026 - mae: 0.0412 - mape: 21831.5781 - mse: 0.0026 - val_loss: 9.1221e-04 - val_mae: 0.0246 - val_mape: 5.3544 - val_mse: 9.1221e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013 - mae: 0.0284 - mape: 15470.6299 - mse: 0.0013 - val_loss: 7.2216e-04 - val_mae: 0.0214 - val_mape: 4.6561 - val_mse: 7.2216e-04
Ep

Processando combinações:  14%|████████▍                                                  | 1/7 [00:44<04:24, 44.10s/it]

model_1: Processando ação: PETR4, combination: 2...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0700 - mae: 0.2137 - mape: 54508.1797 - mse: 0.0700 - val_loss: 0.0048 - val_mae: 0.0580 - val_mape: 12.6873 - val_mse: 0.0048
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0066 - mae: 0.0656 - mape: 17293.6484 - mse: 0.0066 - val_loss: 0.0032 - val_mae: 0.0466 - val_mape: 10.0382 - val_mse: 0.0032
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0031 - mae: 0.0446 - mape: 203070.1719 - mse: 0.0031 - val_loss: 0.0017 - val_mae: 0.0341 - val_mape: 7.3585 - val_mse: 0.0017
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0011 - mae: 0.0263 - mape: 27948.1582 - mse: 0.0011 - val_loss: 7.8893e-04 - val_mae: 0.0227 - val_mape: 4.9885 - val_mse: 7.8893e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.2196e-04 - mae: 0.0156 - mape: 38739.8477 - mse: 4.2196e-04 - val_loss: 4.8897e-04 - val_mae: 0.0167 - val_mape: 3.8507 - val_mse: 4.

...

Processando combinações:  71%|██████████████████████████████████████████▏                | 5/7 [03:39<01:28, 44.13s/it]

model_1: Processando ação: PETR4, combination: 6...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1073 - mae: 0.2673 - mape: 16898.5742 - mse: 0.1073 - val_loss: 0.0103 - val_mae: 0.0847 - val_mape: 18.4814 - val_mse: 0.0103
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0133 - mae: 0.0922 - mape: 22729.3828 - mse: 0.0133 - val_loss: 0.0065 - val_mae: 0.0664 - val_mape: 14.3183 - val_mse: 0.0065
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0053 - mae: 0.0579 - mape: 128651.3047 - mse: 0.0053 - val_loss: 0.0026 - val_mae: 0.0413 - val_mape: 9.0817 - val_mse: 0.0026
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0014 - mae: 0.0300 - mape: 33960.8711 - mse: 0.0014 - val_loss: 8.9175e-04 - val_mae: 0.0231 - val_mape: 5.3170 - val_mse: 8.9175e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.4498e-04 - mae: 0.0158 - mape: 1089.4875 - mse: 4.4498e-04 - val_loss: 4.7242e-04 - val_mae: 0.0157 - val_mape: 3.7812 - val_mse: 4.7

Processando combinações:  86%|██████████████████████████████████████████████████▌        | 6/7 [04:32<00:47, 47.23s/it]

model_1: Processando ação: PETR4, combination: 7...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.1045 - mae: 0.2681 - mape: 30911.8066 - mse: 0.1045 - val_loss: 0.0044 - val_mae: 0.0554 - val_mape: 12.2622 - val_mse: 0.0044
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0046 - mae: 0.0541 - mape: 912.7846 - mse: 0.0046 - val_loss: 0.0023 - val_mae: 0.0381 - val_mape: 8.5794 - val_mse: 0.0023
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013 - mae: 0.0276 - mape: 24839.5566 - mse: 0.0013 - val_loss: 0.0013 - val_mae: 0.0270 - val_mape: 6.3906 - val_mse: 0.0013
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.9917e-04 - mae: 0.0188 - mape: 1364.9385 - mse: 5.9917e-04 - val_loss: 0.0011 - val_mae: 0.0244 - val_mape: 5.8530 - val_mse: 0.0011
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.0432e-04 - mae: 0.0169 - mape: 3790.5342 - mse: 5.0432e-04 - val_loss: 9.5487e-04 - val_mae: 0.0228 - val_mape: 5.5260 - val_mse: 9.5487e-

Processando combinações: 100%|███████████████████████████████████████████████████████████| 7/7 [05:28<00:00, 46.88s/it]


In [21]:
results_df

,Stock,Description,Model,Loss,MSE,RMSE,MAE,MAPE,R²,Acurácia
0,PETR4,Stock Data,model_1,0.000175,0.000325,0.018041,0.112509,27.926719,0.967746,0.514604
1,VALE3,Stock Data,model_1,0.000084,0.000913,0.030208,0.212717,38.588257,0.975416,0.413074
2,BBDC4,Stock Data,model_1,0.000354,0.000519,0.022776,0.146503,27.538990,0.970115,0.442225
3,ITUB4,Stock Data,model_1,0.000189,0.000415,0.020370,0.137782,22.521545,0.972841,0.522949
4,PETR4,Stock Data + Google News,model_1,0.000174,0.000324,0.017988,0.112083,27.824612,0.967935,0.518776
5,VALE3,Stock Data + Google News,model_1,0.000095,0.001864,0.043173,0.209322,37.867155,0.949784,0.312935
6,BBDC4,Stock Data + Google News,model_1,0.000349,0.000540,0.023248,0.145978,27.376342,0.968863,0.440799
7,ITUB4,Stock Data + Google News,model_1,0.000180,0.000404,0.020104,0.137936,22.505104,0.973547,0.528512
8,PETR4,Stock Data + Twitter,model_1,0.000171,0.000318,0.017828,0.112189,27.938379,0.968503,0.539638
9,VALE3,Stock Data + Twitter,model_1,0.000096,0.000684,0.026154,0.213767,39.092867,0.981572,0.436718


In [22]:
!pip install openpyxl

In [23]:
results_df.to_excel(f'Estudos_IEEE_{name_model}.xlsx', index=False, engine='openpyxl')